In [996]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import json
import geopandas as gpd

Todo:
1. Dropdown menu
2. Borough bar chart (req 1)
3. Neighborhood bar chart (req 1,2)
4. Map of NYC (req 1, 2*/3*)
5. Scatterplots (req 1, 2*/3*)

In [997]:
# load in the data csv files
tree_df = pd.read_csv('./final_data/trees.csv')
borough_df = pd.read_csv('./final_data/boroughs.csv')
neighborhood_df = pd.read_csv('./final_data/UHF.csv')
borough_df.rename(columns={"Average Status": "Average Tree Status","Average Diameter": "Average Tree Diameter",  "Average Summer Surface Temperature":"Average Summer Surface Temperature ºF", "Average Fine Particle Pollution": "Average Fine Particle Pollution mcg/m3", "Average Ozone Pollution": "Average Ozone Pollution ppb"}, inplace=True)
neighborhood_df.rename(columns={"Average Status": "Average Tree Status","Average Diameter": "Average Tree Diameter",  "Average Summer Surface Temperature":"Average Summer Surface Temperature ºF", "Average Fine Particle Pollution": "Average Fine Particle Pollution mcg/m3", "Average Ozone Pollution": "Average Ozone Pollution ppb"}, inplace=True)
tree_df = tree_df.rename(columns={'Status':'Tree Status'})
tree_df
suffix = {
        'Number of Trees': 'Trees',
        'Average Tree Diameter': "Inches",
        'Average Tree Status': 'Status Level',
        "Average Summer Surface Temperature ºF": "ºF",
        'Average Fine Particle Pollution mcg/m3': "mcg/m3", 
        'Average Ozone Pollution ppb': "ppb"
    }

FileNotFoundError: [Errno 2] No such file or directory: './final_data/trees.csv'

In [ ]:

with open('./geo_data/borough.geo.json', 'r') as file:
    boroughs = json.load(file)
with open('./geo_data/UHF34.geo.json', 'r') as file:
    nbds = json.load(file)
borough_df['BoroName'] = borough_df['Borough']
del nbds["features"][0]
boro_gdf = gpd.GeoDataFrame.from_features(boroughs["features"])
uhf_gdf = gpd.GeoDataFrame.from_features(nbds["features"])
uhf_gdf.set_index('GEOCODE', inplace=True)
uhf_bounds=uhf_gdf.bounds
boro_gdf.set_index('BoroName', inplace=True)
boro_bounds=boro_gdf.bounds

In [ ]:
def get_bounds(b_list, x_to_y=0, scale_x=1.1):
    b_box = 0
    if not isinstance(b_list, pd.core.frame.DataFrame):
        b_box = [b_list['minx'], 
            b_list['maxx'],
            b_list['miny'],
            b_list['maxy']]
    else:
        b_box = [min(b_list['minx']), 
            max(b_list['maxx']),
            min(b_list['miny']),
            max(b_list['maxy'])]
    center = [(b_box[1]+b_box[0])/2, (b_box[3]+b_box[2])/2]
    width = b_box[1]-b_box[0]
    height = b_box[3]-b_box[2]
    ratio = width/height
    scale = scale_x
    scale_x = 1
    scale_y = 1
    if x_to_y == 0:
        ()
    elif x_to_y < ratio:
        scale_y = ratio/x_to_y
    elif x_to_y > ratio:
        scale_x = x_to_y/ratio
    new_left = center[0] - width/2 * scale * scale_x
    new_right = center[0] + width/2 * scale * scale_x
    new_top = center[1] + height/2 * scale * scale_y   
    new_bottom = center[1] - height/2 * scale * scale_y
    return {
        "lataxis":{
            "range":[new_bottom,new_top]
        },
        "lonaxis":{
            "range":[new_left,new_right]
        }
    }

In [ ]:
# make the borough graph
fig = px.bar(y=borough_df["Borough"], x=borough_df["Number of Trees"], orientation='h')
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig.update_traces(hovertemplate='%{y}<br>%{x} '+ suffix['Number of Trees'])
borough_bar = go.Figure(data=fig)

In [ ]:
# make the neighborhood graph
fig = px.bar(x=neighborhood_df["UHF34 Neighborhood Name"], y=neighborhood_df["Number of Trees"], orientation='h')
fig.update_layout(margin=dict(l=100, r=15, t=20, b=20))
fig.update_traces(hovertemplate='%{y}<br>%{x} '+ suffix['Number of Trees'])
neighborhood_bar = go.Figure(data=fig)

In [ ]:
# TODO: make the map ???
dataname = 'Number of Trees'
fig = go.Figure(data=go.Choropleth(
        geojson=boroughs, # for UHF use `nbds`
        colorscale="Viridis",
        locations=borough_df['BoroName'], #for UHF use `neighborhood_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.BoroName", #for UHF use "properties.UHF" 
        z=borough_df[dataname], # for UHF use `neighborhood_df[dataname]`
        zmin=0,
        zmax=borough_df[dataname].max()
        ))

fig.update_layout(geo = dict(
        landcolor = "rgb(212, 212, 212)",
        resolution = 50,
        lonaxis = dict(
            showgrid = False,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = False,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    margin=dict(l=20, r=20, t=50, b=20)
    )

map = go.Figure(data=fig)
map.update_geos(fitbounds="locations")
()


()

In [ ]:
# make the scatterplots
fig1 = px.scatter(neighborhood_df, x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Fine Particle Pollution mcg/m3"], custom_data=['UHF34 Code','Borough'])
fig2 = px.scatter(neighborhood_df, x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Tree Diameter"], custom_data=['UHF34 Code','Borough'])
fig3 = px.scatter(neighborhood_df, x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Summer Surface Temperature ºF"], custom_data=['UHF34 Code','Borough'])
fig1.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig2.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig3.update_layout(margin=dict(l=20, r=20, t=20, b=20))
metric1_scatter = go.Figure(data=fig1)
metric2_scatter = go.Figure(data=fig2)
metric3_scatter = go.Figure(data=fig3)

neighborhood_df.head()

,Unnamed: 0,Average Tree Status,Average Tree Diameter,Borough,UHF34 Code,Number of Trees,Average Summer Surface Temperature ºF,Average Fine Particle Pollution mcg/m3,Average Ozone Pollution ppb,UHF34 Neighborhood Name
0,0,2.709569,13.385147,Brooklyn,209,14327,94.8,7.671429,31.921429,Bensonhurst - Bay Ridge
1,1,2.690632,14.122453,Queens,404406,36177,97.6,7.600000,31.471429,Bayside Little Neck-Fresh Meadows
2,2,2.705653,11.143644,Brooklyn,203,19973,98.5,8.214286,30.542857,Bedford Stuyvesant - Crown Heights
3,3,2.676278,13.192261,Brooklyn,206,21398,98.6,7.907143,31.264286,Borough Park
4,4,2.653037,14.679529,Brooklyn,208,19100,95.9,7.578571,33.800000,Canarsie - Flatlands


In [ ]:
# Dash app setup
app = Dash(__name__) # TODO: add stylesheet
app.layout = html.Div([
    html.Div(
        [
        html.H1('ArborAirAnalytics'),
        dcc.Dropdown(
            ['Number of Trees', 'Average Tree Status', 'Average Tree Diameter', 'Average Fine Particle Pollution mcg/m3', 'Average Ozone Pollution ppb', 'Average Summer Surface Temperature ºF'],
            'Number of Trees',
            id='metric-dropdown'
        ),
        ],
        id='header-div'
    ),
    html.Div(
        [
            html.Div(
                [
                    html.Div(
                        [
                            dcc.Graph(id='borough-bar', figure=borough_bar),
                            html.Div(
                                [html.Button('Deselect Borough', id='borough-deselect-button')],
                                className='center'
                            )
                            
                        ],
                        id='borough-div',
                        className='column'
                    ),
                    html.Div(
                        [
                            dcc.Graph(id='neighborhood-bar', figure=neighborhood_bar),
                            html.Div(
                                [html.Button('Deselect Neighborhood', id='neighborhood-deselect-button')],
                                className='center'
                            )
                        ],
                        id='neighborhood-div',
                        className='column'
                    )
                ],
                id='bar-div',
                className='row'
            ),
            html.Div(
                [
                    dcc.Graph(id='map', figure=map)
                ],
                id='map-div',
                className='column'
            ),
            html.Div(
                [
                    dcc.Graph(id='metric1-scatter', className='scatter', figure=metric1_scatter),
                    dcc.Graph(id='metric2-scatter', className='scatter', figure=metric2_scatter),
                    dcc.Graph(id='metric3-scatter', className='scatter', figure=metric3_scatter)
                ],
                id='scatter-div',
                className='column'
            )
        ],
        id='visualization-div',
        className='row'
    )
],
)

In [ ]:
def update_borough_bar(chosen_metric, clicked_borough):
    
    # Create the bar chart
    fig = px.bar(
        y=borough_df["Borough"], 
        x=borough_df[chosen_metric], 
        title=f'{chosen_metric} by Borough', 
        orientation='h', 
        

    )


    # Color the clicked borough
    if clicked_borough:
        colors = ['purple' if borough == clicked_borough else '#636EFA' for borough in borough_df["Borough"]]
        fig.update_traces(marker_color=colors)        

    # Update axes and layout
    fig.update_yaxes(title="Borough")
    fig.update_xaxes(title=chosen_metric)
    fig.update_layout(
        yaxis={'categoryorder':'total ascending'}, 
        margin=dict(l=20, r=20, t=50, b=20)
    )
    fig.update_traces(hovertemplate='%{y}<br>%{x} '+ suffix[chosen_metric])        

    borough_bar = go.Figure(data=fig)
    return borough_bar

In [ ]:
def update_neighborhood_bar(chosen_metric, clicked_borough, clicked_neighborhood):

    if clicked_borough != None:
        neighborhood_df_filtered = neighborhood_df[neighborhood_df["Borough"] == clicked_borough]
    else:
        neighborhood_df_filtered = neighborhood_df
    
    # TODO: if borough is selected, filter neighborhood_df_filtered by borough
    fig = px.bar(
        y=neighborhood_df_filtered["UHF34 Neighborhood Name"], 
        x=neighborhood_df_filtered[chosen_metric], 
        title=f'{chosen_metric} by Neighborhood', 
        orientation='h', 
       
    )

    # Color the clicked borough
    if clicked_neighborhood:
        colors = ['purple' if neighborhood == clicked_neighborhood else '#636EFA' for neighborhood in neighborhood_df_filtered["UHF34 Neighborhood Name"]]
        fig.update_traces(marker_color=colors)        

    fig.update_yaxes(title="Neighborhood")
    fig.update_xaxes(title=chosen_metric)
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    fig.update_traces(hovertemplate='%{y}<br>%{x} '+ suffix[chosen_metric])    
    neighborhood_bar = go.Figure(data=fig)
    return neighborhood_bar

In [ ]:
def update_map(chosen_metric, clicked_borough, clicked_nbd):
    min_val = 0
    if chosen_metric == 'Average Summer Surface Temperature ºF':
        min_val=90
    elif chosen_metric == 'Average Ozone Pollution ppb':
        min_val = 20
    elif chosen_metric == 'Average Fine Particle Pollution mcg/m3':
        min_val = 4
    elif chosen_metric == 'Average Tree Status':
        min_val = 2
    elif chosen_metric == 'Average Tree Diameter':
        min_val = 5
    
    map_ratio = (60-20)/(140-55)
    map_ratio = 285/385
    if clicked_borough == None:
        fig = go.Figure(data=go.Choropleth(
        geojson=boroughs, # for UHF use `nbds`
        colorscale="Blues",
        locations=borough_df['BoroName'], #for UHF use `neighborhood_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.BoroName", #for UHF use "properties.UHF" 
        z=borough_df[chosen_metric], # for UHF use `neighborhood_df[dataname]`
        customdata=borough_df['BoroName'],
        hovertemplate='%{customdata}<br>'+chosen_metric+':%{z}',
        zmin=min_val,
        zmax=borough_df[chosen_metric].max()
        ))

        fig.update_layout(geo = dict(
                landcolor = "rgb(212, 212, 212)",
                resolution = 50,
                lonaxis = dict(
                    showgrid = False,
                    gridwidth = 0.5,
                    range= [ -140.0, -55.0 ],
                    dtick = 5
                ),
                lataxis = dict (
                    showgrid = False,
                    gridwidth = 0.5,
                    range= [ 20.0, 60.0 ],
                    dtick = 5
                )
            ),
            margin=dict(l=20, r=20, t=70, b=20),
            title=f'{chosen_metric} by Borough<br> '
        )
        if clicked_nbd is not None:
             fig.update_layout(title=f'{chosen_metric} by Borough,<br> Trees in {clicked_nbd}')
        map = go.Figure(data=fig)
        map.update_geos(get_bounds(boro_bounds, map_ratio))
    elif clicked_borough != None:
        fig = go.Figure(data=go.Choropleth(
        geojson=nbds, # for UHF use `nbds`
        colorscale="Blues",
        locations=neighborhood_df['UHF34 Code'], #for UHF use `neighborhood_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.UHF", #for UHF use "properties.UHF"
        # customdata=neighborhood_df['UHF34 Code'],
        customdata=neighborhood_df[['UHF34 Code','UHF34 Neighborhood Name']],
        hovertemplate='UHF34 Code:%{customdata[0]}<br>{customdata[1]}<br>'+chosen_metric+':%{z}',
        z=neighborhood_df[chosen_metric], # for UHF use `neighborhood_df[dataname]`
        zmin=min_val,
        zmax=neighborhood_df[chosen_metric].max()
        ))

        fig.update_layout(geo = dict(
                landcolor = "rgb(212, 212, 212)",
                resolution = 50,
                lonaxis = dict(
                    showgrid = False,
                    gridwidth = 0.5,
                    range= [ -140.0, -55.0 ],
                    dtick = 5
                ),
                lataxis = dict (
                    showgrid = False,
                    gridwidth = 0.5,
                    range= [ 20.0, 60.0 ],
                    dtick = 5
                )
            ),
            margin=dict(l=20, r=20, t=70, b=20),
            title=f'{chosen_metric} by Neighborhood in {clicked_borough}<br>'
        )
        if clicked_nbd is not None:
             fig.update_layout(title=f'{chosen_metric} by Neighborhood in {clicked_borough},<br> Trees in {clicked_nbd}')
        map = go.Figure(data=fig)
        map.add_trace(
            go.Choropleth(
                name=clicked_borough,
                geojson=nbds, # for UHF use `nbds`
                colorscale="Blues",
                locations=neighborhood_df[neighborhood_df['Borough']==clicked_borough]['UHF34 Code'], #for UHF use `UHF_df['UHF34 Code']`
                locationmode="geojson-id",
                customdata=neighborhood_df[neighborhood_df['Borough']==clicked_borough][['UHF34 Code','UHF34 Neighborhood Name']],
                hovertemplate='UHF34 Code:%{customdata[0]}<br>{customdata[1]}<br>'+chosen_metric+':%{z}',
                featureidkey="properties.UHF", #for UHF use "properties.UHF"
                z=neighborhood_df[chosen_metric], # for UHF use `UHF_df[dataname]`
                zmin=min_val,
                zmax=max(neighborhood_df[chosen_metric]),
                marker=dict(
                    line=dict(
                        width=2,
                        color="gold"
                    )
                )
            )
        )
        map.update_geos(get_bounds(boro_bounds.loc[clicked_borough], map_ratio))
        map.update_layout(margin=dict(l=20, r=20, t=70, b=20))
    if clicked_nbd is not None:
        code = neighborhood_df[neighborhood_df['UHF34 Neighborhood Name']==clicked_nbd]['UHF34 Code'].iloc[0]
        filtered_trees = tree_df[tree_df['UHF34 Code']==code]
        poor_trees = filtered_trees[filtered_trees['Tree Status']==1]
        dead_trees = filtered_trees[filtered_trees['Tree Status']==0]
        okay_trees = filtered_trees[filtered_trees['Tree Status']>1]
        
        map.add_trace(
            go.Scattergeo(
                name=f'Show {len(okay_trees)} Healthy Trees',
                lon = okay_trees['Long'],
                lat = okay_trees['Lat'],
                mode = 'markers',
                customdata=okay_trees['Diameter'],
                hovertemplate='<b>Diameter:%{customdata}</b>',
                marker = dict(
                    size = 8,
                    # opacity = 0.7,
                    symbol = 'triangle-up',
                    color = 'green'
                ),
                visible='legendonly'
            )
        )
        map.add_trace(
            go.Scattergeo(
                name=f'Show {len(poor_trees)} Unhealthy Trees',
                lon = poor_trees['Long'],
                lat = poor_trees['Lat'],
                mode = 'markers',
                customdata=poor_trees['Diameter'],
                hovertemplate='<b>Diameter:%{customdata}</b>',
                marker = dict(
                    size = 8,
                    # opacity = 0.7,
                    symbol = 'triangle-up',
                    color = 'gold'
                ),
                visible='legendonly'
            )
        )
        map.add_trace(
            go.Scattergeo(
                name=f'Show {len(dead_trees)} Dead Trees',
                lon = dead_trees['Long'],
                lat = dead_trees['Lat'],
                customdata=dead_trees['Diameter'],
                hovertemplate='<b>Diameter:%{customdata}</b>',
                mode = 'markers',
                marker = dict(
                    size = 8,
                    # opacity = 0.7,
                    symbol = 'triangle-up',
                    color = 'red'
                ),
                visible='legendonly'
            )
        )
        map.update_geos(get_bounds(uhf_bounds.loc[code], map_ratio))

    return map

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1548730892.py, line 62)

In [ ]:
def update_scatters(chosen_metric, clicked_borough, clicked_neighborhood, other_metrics):
    
    highlighted_df = neighborhood_df[neighborhood_df["Borough"] == clicked_borough] if clicked_borough else None
    # Handle clicked neighborhood logic
    highlighted_neighborhood_df = (
        neighborhood_df[neighborhood_df["UHF34 Neighborhood Name"] == clicked_neighborhood]
        if clicked_neighborhood else None
    )

    scatters = []
    for other_metric in other_metrics:
        if not other_metric or not chosen_metric:
            # Provide a default empty plot
            fig = px.scatter(pd.DataFrame({'x': [], 'y': []}), x='x', y='y')

        fig = go.Figure(data=go.Scatter(
            name="Other",
            x=neighborhood_df[chosen_metric],
            y=neighborhood_df[other_metric],
            mode='markers',
            # marker=dict(color='blue'),
            customdata=neighborhood_df[['UHF34 Code','Borough']],
            hovertemplate='%{customdata[1]}, %{customdata[0]}<br>x:%{x} '+ suffix[chosen_metric]+'<br>y:%{y} ' + suffix[other_metric]
        ))
        fig.update_layout(
            xaxis_title=chosen_metric, 
            yaxis_title=other_metric, 
            margin=dict(l=20, r=20, t=20, b=20)
        )

        # Add highlighted points if a borough is clicked
        if highlighted_df is not None:
            fig.add_trace(
                go.Scatter
                (
                x=highlighted_df[chosen_metric],
                y=highlighted_df[other_metric],
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}",
                customdata=highlighted_df[['UHF34 Code','Borough']],
                hovertemplate='%{customdata[1]}, %{customdata[0]}<br>x:%{x} '+ suffix[chosen_metric]+'<br>y:%{y} ' + suffix[other_metric]
            ))
        
        # Add highlighted points if a neighborhood is clicked
        if highlighted_neighborhood_df is not None:
            fig.add_trace(
                go.Scatter
                (
                x=highlighted_neighborhood_df[chosen_metric],
                y=highlighted_neighborhood_df[other_metric],
                mode='markers',
                marker=dict(color='green'),
                name=f"{clicked_neighborhood}",
                customdata=highlighted_neighborhood_df[['UHF34 Code','Borough']],
                hovertemplate='%{customdata[1]}, %{customdata[0]}<br>x:%{x} '+ suffix[chosen_metric]+'<br>y:%{y} ' + suffix[other_metric]
            ))
        # fig.update_traces(hovertemplate='%{customdata[1]}, %{customdata[0]}<br>x:%{x} '+ suffix[chosen_metric]+'<br>y:%{y} ' + suffix[other_metric])
        # Update layout and return the plot
        scatters.append(fig)
    return scatters



In [ ]:
@app.callback(
    Output("borough-bar", "figure"),
    Output('borough-bar', 'clickData'),
    Output('borough-deselect-button', 'n_clicks'),
    Output("neighborhood-bar", "figure"),
    Output('neighborhood-bar', 'clickData'),
    Output("neighborhood-deselect-button", 'n_clicks'),
    Output("map", "figure"),
    Output("metric1-scatter", "figure"),
    Output("metric2-scatter", "figure"),
    Output("metric3-scatter", "figure"),
    [
        Input("metric-dropdown", "value"),
        Input('borough-bar', 'clickData'),
        Input('borough-deselect-button', 'n_clicks'),
        Input('neighborhood-bar', 'clickData'),
        Input('neighborhood-deselect-button', 'n_clicks')
    ]
)
def update_dash(chosen_metric, clicked_borough_bar, borough_deselect_clicks, clicked_nbd_bar, nbd_deselect_clicks):
    # If the borough is clicked, store the clicked borough
    borough_clickdata = clicked_borough_bar
    clicked_borough = borough_clickdata['points'][0]['y'] if borough_clickdata else None
    if borough_deselect_clicks and borough_deselect_clicks > 0:
         clicked_borough = None
         borough_clickdata = None
    
    nbd_clickdata = clicked_nbd_bar
    clicked_nbd = nbd_clickdata['points'][0]['y'] if nbd_clickdata else None
    if nbd_deselect_clicks and nbd_deselect_clicks > 0:
         clicked_nbd = None
         nbd_clickdata = None         

    borough_bar = update_borough_bar(chosen_metric, clicked_borough)
    neighborhood_bar = update_neighborhood_bar(chosen_metric, clicked_borough, clicked_nbd)
    map = update_map(chosen_metric, clicked_borough, clicked_nbd)

    tree_metrics = ['Number of Trees', 'Average Tree Status', 'Average Tree Diameter']
    env_metrics = ['Average Fine Particle Pollution mcg/m3', 'Average Ozone Pollution ppb', 'Average Summer Surface Temperature ºF']

    other_metrics = env_metrics if chosen_metric in tree_metrics else tree_metrics

    [metric1_scatter, metric2_scatter, metric3_scatter] = update_scatters(chosen_metric, clicked_borough, clicked_nbd, other_metrics)

    return borough_bar, borough_clickdata, None, neighborhood_bar, nbd_clickdata, None, map, metric1_scatter, metric2_scatter, metric3_scatter

if __name__ == '__main__':
    app.run_server(debug=True)